# O objetivo da camada bronze é:

- Armazena dados brutos das fontes sem modificações. 
- Repositório confiável para todos os dados ingeridos, permitindo auditoria e reprocessamento. 
- Importante garantir que a estrutura da camada Bronze reflita fielmente a estrutura da fonte de dados

In [1]:
import pandas as pd
import duckdb

# Extração

- Tabela Consumo
- Tabela Clima
- Tabela Clientes

In [2]:
# Caminho do banco
db_path = "../../data/duckdb/database.duckdb"

# Conexão com o banco DuckDB
con = duckdb.connect(db_path)

folder_data = '../../data/dados_brutos'
clima_df = pd.read_csv(f'{folder_data}/clima.csv',parse_dates=['date'])
consumo_df = pd.read_csv(f'{folder_data}/consumo.csv',parse_dates=['date'])
clientes_df = pd.read_csv(f'{folder_data}/clientes.csv')

In [3]:
# Cria schema 'bronze' se não existir
con.execute("CREATE SCHEMA IF NOT EXISTS bronze")

# Cria schema 'outputs' se não existir
con.execute("CREATE SCHEMA IF NOT EXISTS output")

# Cria schema 'feature' se não existir
con.execute("CREATE SCHEMA IF NOT EXISTS feature")

# Carregamento

In [4]:
# Cria tabelas dentro do schema bronze
con.execute("""
CREATE TABLE IF NOT EXISTS bronze.clientes (
    client_id VARCHAR,
    region VARCHAR
)
""")

con.execute("""
CREATE TABLE IF NOT EXISTS bronze.consumo (
    client_id VARCHAR,
    date DATE,
    consumption_kwh DOUBLE
)
""")

con.execute("""
CREATE TABLE IF NOT EXISTS bronze.clima (
    region VARCHAR,
    date DATE,
    temperature DOUBLE,
    humidity DOUBLE
)
""")

In [5]:
# Limpa dados se as tabelas já existirem
con.execute("DELETE FROM bronze.clientes")
con.execute("DELETE FROM bronze.consumo")
con.execute("DELETE FROM bronze.clima")

In [6]:
# Registra como tabelas temporárias
con.register("clientes_df", clientes_df)
con.register("consumo_df", consumo_df)
con.register("clima_df", clima_df)

In [7]:
# Insere os dados nas tabelas bronze
con.execute("INSERT INTO bronze.clientes SELECT * FROM clientes_df")
con.execute("INSERT INTO bronze.consumo SELECT * FROM consumo_df")
con.execute("INSERT INTO bronze.clima SELECT * FROM clima_df")

In [8]:
con.close()
print("✔ Dados inseridos com sucesso na camada bronze do DuckDB.")

✔ Dados inseridos com sucesso na camada bronze do DuckDB.
